In [ ]:
import numpy as np
import sqlite3
import PIL.Image
import glob
import os

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import optimizers

import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
IMAGE_SIZE = (400, 150, 3)
RESIZED_SIZE = (100, 50, 3)
RESIZED_SIZE_PIL = (RESIZED_SIZE[1], RESIZED_SIZE[0], RESIZED_SIZE[2])
DATASET_PATH = "./data/"
DATASET_PATH = os.path.abspath(DATASET_PATH)

print(DATASET_PATH)


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32
SHUFFLE_SIZE = 64
LIMIT = 1000


def customGenerator():
    data = (
        sqlite3.connect(f"{DATASET_PATH}/index.db")
        .execute(f"SELECT uuid, model from data LIMIT {LIMIT}")
        .fetchall()
    )

    for uuid, model in data:
        img = tf.io.read_file(f"{DATASET_PATH}/{uuid}.jpg")
        img = tf.image.decode_jpeg(img, channels=IMAGE_SIZE[2])
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, RESIZED_SIZE[:-1])

        label = tf.convert_to_tensor(model, dtype=tf.uint8)

        yield img, label


def cutout(image, label):
    img = tfa.image.random_cutout(image, (6, 6), constant_values=1)
    return (img, label)


def rotate(image, label):
    img = tfa.image.rotate(image, tf.constant(np.pi))
    return (img, label)


def set_shapes(image, label):
    image.set_shape(RESIZED_SIZE)
    label.set_shape([])
    return image, label


dataset = tf.data.Dataset.from_generator(generator=customGenerator, output_types=(tf.float32, tf.uint8))

(dataset_length,) = sqlite3.connect(f"{DATASET_PATH}/index.db").execute("SELECT count(uuid) from data").fetchone()
dataset_length = min(dataset_length, LIMIT)

print(f"dataset_length = {dataset_length}")
print(f"batch size = {BATCH_SIZE}")
print(f"number of batchs = {dataset_length // BATCH_SIZE}")

print()

train_size = int(0.8 * dataset_length / BATCH_SIZE)
print(f"train_size = {train_size}")
print(f"validation_size = {dataset_length - train_size}")

dataset = (
    dataset.shuffle(SHUFFLE_SIZE)
    .map(set_shapes)
    .batch(BATCH_SIZE)
    # .map(cutout)
    .prefetch(AUTOTUNE)
)

dataset_train = dataset.take(train_size)
dataset_validate = dataset.skip(train_size)


In [ ]:
model = Sequential(
    [
        InputLayer(input_shape=RESIZED_SIZE),
        Conv2D(32, 3, activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, 3, activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(92, 3, activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(250, activation="relu"),
        Dense(4, activation="softmax"),
    ]
)

model.summary()

In [ ]:
adam = optimizers.Adam(learning_rate=7e-6)
model.compile(optimizer=adam, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(dataset_train, validation_data=dataset_validate, epochs=5, batch_size=BATCH_SIZE)


In [ ]:
def plot_training_analysis():
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    epochs = range(len(loss))

    plt.plot(epochs, acc, "b", linestyle="--", label="Training acc")
    plt.plot(epochs, val_acc, "g", label="Validation acc")
    plt.title("Training and validation accuracy")
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, "b", linestyle="--", label="Training loss")
    plt.plot(epochs, val_loss, "g", label="Validation loss")
    plt.title("Training and validation loss")
    plt.legend()

    plt.show()


plot_training_analysis()


In [ ]:
# Save the weights
# model.save('models/rot_25e')